In [1]:
from spark.config.config_set import UniversalConfigSet
from spark.model.gaussian_model import GaussianModel
from spark.model.training_data import TrainingData
import numpy as np

In [2]:
training_data = TrainingData()
config_set = UniversalConfigSet(10, 1024 * 10)
model = GaussianModel(training_data, config_set)

In [3]:
training_sample_1 = {
    "spark.executor.memory": 1024 * 5,
    "spark.sql.shuffle.partitions": 100,
    "spark.executor.cores": 4,
    "spark.driver.memory": 1024
}
training_sample_2 = {
    "spark.executor.memory": 1024 * 10,
    "spark.sql.shuffle.partitions": 400,
    "spark.executor.cores": 8,
    "spark.driver.memory": 1024 * 3
}
model.add_sample_to_train_data(training_sample_1, 12)
model.add_sample_to_train_data(training_sample_2, 4)
print model.get_best_config()

In [6]:
normalized_values = model.get_sampled_configs()
model.train()

In [7]:
# DEBUG Paragraph
# Mean Calculation
config = model.get_sampled_configs()[0]
print config
print model.beta
term1 = np.dot(config, model.beta)
print "term1 %s" % term1
term2 = model.training_out - np.dot(model.get_training_params(), model.beta)
print "term2 %s" % term2
term31 = model.get_correlation_with_train_data(config).transpose()
term32 = np.linalg.inv(model.get_training_pairwise_correlation())
term3 = np.dot(term31, term32)
print "term3 %s" % term3
term4 = np.dot(term3, term2)
print "term4 %s" % term4
print term1 + term4

[0.8461538461538461, 0.1875, -0.0, -0.0]
[[1.e-06]
 [1.e-06]
 [1.e-06]
 [1.e-06]]
term1 [1.03365385e-06]
term2 [[11.99999819]
 [ 3.99999425]]
term3 [[0.93987784 0.03812624]]
term4 [[11.43103713]]
[[11.43103816]]


In [13]:
# DEBUG Paragraph
# Variance calculation
config = model.get_sampled_configs()[0]
corr_with_train_data = model.get_correlation_with_train_data(config)
corr_pairwise_train_data = model.get_training_pairwise_correlation()
print corr_with_train_data
print corr_pairwise_train_data
term1 = np.dot(corr_with_train_data.transpose(), np.linalg.inv(corr_pairwise_train_data))
term2 = np.dot(term1, corr_with_train_data)
term3 = 1 - term2
print term3
print np.linalg.det(term3)

[[0.9765291 ]
 [0.94164311]]
[[1.         0.96131309]
 [0.96131309 1.        ]]
[[0.04628063]]
0.046280627946319715


In [6]:
#DEBUG Paragraph
import sys
from spark.discretizer.normalizer import ConfigNormalizer
normalized_values = model.get_sampled_configs()
best_config_value = None
best_config = {}
best_out = sys.maxint
# for config in list(itertools.product(*normalized_values)):
for config in normalized_values:
    out = model.predict(config)
    if out < best_out:
        best_out = out
        best_config_value = config
print best_out
print best_config_value
# model.normalizer.denormalize_config(best_config_value)
i = 0
for param in model.normalizer.get_params():
    print ConfigNormalizer.denormalize_value(param, best_config_value[i])

[4.62752883e-205]
[0.02564102564102564, 0.5, 0.6666666666666666, 0.625]
60.0
1131.0
276.0
2.0


In [12]:
#DEBUG Paragraph
import sys
from spark.discretizer.normalizer import ConfigNormalizer

normalized_values = model.get_sampled_configs()
best_config_value = None
best_config = {}
best_out = sys.maxint
# for config in list(itertools.product(*normalized_values)):
for config in normalized_values:
    out = model.predict(config)
    if out < best_out:
        best_out = out
        best_config_value = config

denorm_best_config = model.normalizer.denormalize_config(best_config_value)
print denorm_best_config
for param in model.normalizer.get_params():
    print param.get_name() 

[60.0, 5017.0, 768.0, 7.0]
spark.sql.shuffle.partitions
spark.executor.memory
spark.driver.memory
spark.executor.cores
